In [ ]:
from random import shuffle

# Define a class representing a node in the search tree
class SearchNode:
    def __init__(self, state, parent, cost=1):
        self.state = state
        self.parent = parent
        self.cost = cost

    # Check if the goal state is reached
    def goal_reached(self, searcher):
        return self.state.goal_reached(searcher)

    # Calculate the difference between the current state and the goal state
    def difference(self, goal):
        return self.state.difference(goal)

    # Calculate the total cost of reaching this node
    def total_cost(self):
        cost = 0
        n = self
        while n.parent is not None:
            cost += n.cost
            n = n.parent
        return cost

    # Calculate the evaluation function value for this node
    def evaluation_fn(self, goal):
        return self.difference(goal) + self.total_cost()

    # Get successor nodes in the search tree
    def get_successors(self, searcher):
        successors = [SearchNode(suc_state, self, suc_state.cost_from(self.state)) for suc_state in self.state.get_successors(searcher)]
        return successors

    # Check if the current state is the same as another node's state
    def same_state(self, other):
        return self.state.same_state(other.state)

    # String representation of the node
    def __str__(self):
        return f"Node with state {str(self.state)}"


# Define a class representing a generic search algorithm
class Search:
    def __init__(self):
        self.init_node = None
        self.current_node = None
        self.open = []
        self.closed = set()
        self.successor_nodes = []
        self.goal_state = None

    # Get the goal state
    def get_goal(self):
        return self.goal_state

    # Set the goal state
    def set_goal(self, goal):
        self.goal_state = goal

    # Get the current node in the search
    def get_current_node(self):
        return self.current_node

    # Run the search algorithm
    def run_search(self, init_state, search_method):
        print("\nStarting Search")
        self.init_node = SearchNode(init_state, None)
        self.open = [self.init_node]
        self.closed = set()
        cnum = 1
        while self.open:
            print(f"\nIteration {cnum}")
            self.select_node(search_method)
            print(f"Selected Node: {str(self.current_node.state)}")
            if self.current_node.goal_reached(self):
                return self.report_success()
            self.expand(search_method)
            cnum += 1
        return "Search Fails"

    # Expand the search tree
    def expand(self, search_method):
        self.successor_nodes = self.current_node.get_successors(self)
        self.validate_successors(search_method)
        if search_method == "depth_first":
            self.open.extend(self.successor_nodes[::-1])  # Reverse to implement depth-first behavior
        else:
            self.open.extend(self.successor_nodes)

    # Validate successors to avoid duplicates
    def validate_successors(self, search_method):
        valid_successors = [s for s in self.successor_nodes if s not in self.closed and s not in self.open]
        self.successor_nodes = valid_successors

    # Select the next node based on the search method
    def select_node(self, search_method):
        if search_method == "depth_first":
            self.current_node = self.open.pop()
        elif search_method == "breadth_first":
            self.current_node = self.open.pop(0)
        else:
            raise ValueError(f"Unsupported search method: {search_method}")

    # Report success and print the solution path
    def report_success(self):
        n = self.current_node
        path = [str(n)]
        plen = 1
        while n.parent is not None:
            n = n.parent
            path.insert(0, str(n))
            plen += 1

        print("Search Succeeds")
        print(f"Efficiency {plen / len(self.closed)}")
        print(f"Nodes visited: {len(self.closed) + 1}")
        print("Solution Path:")
        return "\n".join(path)


# Define a generic search state class
class SearchState:
    pass


# Define a specific search state class for the Bingo problem
class BingoState(SearchState):
    def __init__(self):
        # Initialize a grid for the Bingo game
        self.grid = [[-1 for _ in range(9)] for _ in range(18)]
        self.unassigned = [(i, j) for i in range(18) for j in range(9)]
        self.col_sums = [0] * 9

    # Check if the goal state is reached for the Bingo problem
    def goal_reached(self, searcher):
        if self.unassigned:
            return False

        if not self.check_row_sums() or not self.check_column_sums() or not self.check_column_triplets():
            return False

        return True

    # Check if the row sums are valid
    def check_row_sums(self):
        return all(sum(row) == 5 for row in self.grid)

    # Check if the column sums are valid
    def check_column_sums(self):
        for j in range(9):
            if j == 0 and self.col_sums[j] != 9:
                return False
            if j == 8 and self.col_sums[j] != 11:
                return False
            if 0 < j < 8 and self.col_sums[j] != 10:
                return False
        return True

    # Check if the column triplets are valid
    def check_column_triplets(self):
        for j in range(9):
            col_sums = [sum(self.grid[i][j] for i in range(0, 18, 3)) for i in range(3)]
            if j == 0 and col_sums != [1, 1, 1, 2, 2, 2]:
                return False
            if j == 8 and col_sums != [2, 2, 2, 2, 2, 1]:
                return False
            if 0 < j < 8 and col_sums != [2, 2, 1, 2, 2]:
                return False
        return True

    # Get successors for the current state
    def get_successors(self, searcher):
        successors = []
        shuffle(self.unassigned)

        for i, j in self.unassigned:
            for val in [0, 1]:
                state = self.copy()
                state.assign(i, j, val)
                if state.is_consistent(i, j):
                    successors.append(state)

        return successors

    # Check if the assignment is consistent
    def is_consistent(self, i, j):
        row_sum = sum(self.grid[i])
        col_sum = sum(row[j] for row in self.grid)
        return row_sum <= 5 and col_sum <= 11

    # Assign a value to a cell in the grid
    def assign(self, i, j, val):
        self.grid[i][j] = val
        self.unassigned.remove((i, j))
        self.col_sums[j] += val

    # Create a copy of the current state
    def copy(self):
        state = BingoState()
        state.grid = [row[:] for row in self.grid]
        state.unassigned = self.unassigned[:]
        state.col_sums = self.col_sums[:]
        return state

    # Check if the current state is the same as another state
    def same_state(self, other):
        return self.grid == other.grid

    # Get the cost from the previous state (not used in your implementation)
    def cost_from(self, from_state):
        return 1

    # Calculate the difference between the current state and a goal state
    def difference(self, goal):
        return sum(1 for i in range(18) for j in range(9) if self.grid[i][j] != goal.grid[i][j])


# Define a specific search class for the Bingo problem
class BingoSearch(Search):
    pass

# Main entry point
if __name__ == "__main__":
    # Create an initial state for the Bingo problem
    init_state = BingoState()
    # Create a search object for the Bingo problem
    searcher = BingoSearch()
    # Run the depth-first search for the Bingo problem
    solution = searcher.run_search(init_state, 'depth_first')

    # Print the solution if found, otherwise, print a message
    if solution:
        print(solution)
    else:
        print("No solution found.")


In [ ]:
from random import shuffle


class SearchNode:
    def __init__(self, state, parent, cost=1):
        self.state = state
        self.parent = parent
        self.cost = cost

    def goalP(self, searcher):
        return self.state.goalP(searcher)

    def difference(self, goal):
        return self.state.difference(goal)

    def best_path_cost(self):
        cost = 0
        n = self
        while n.parent is not None:
            cost += n.cost
            n = n.parent
        return cost

    def evaluation_fn(self, goal):
        return self.difference(goal) + self.best_path_cost()

    def get_successors(self, searcher):
        slis = self.state.get_successors(searcher)
        nlis = []
        for suc_state in slis:
            n = SearchNode(suc_state, searcher.current_node,
                           suc_state.cost_from(searcher.current_node.state))
            nlis.append(n)
        return nlis

    def same_state(self, n2):
        return self.state.same_state(n2.state)

    def __str__(self):
        return f"Node with state {str(self.state)}"


class Search:
    def __init__(self):
        self.init_node = None
        self.current_node = None
        self.open = []
        self.closed = []
        self.successor_nodes = []
        self.goal_state = None

    def get_goal(self):
        return self.goal_state

    def put_goal(self, goal):
        self.goal_state = goal

    def run_search(self, init_state, g_state, search_method):
        self.goal_state = g_state
        return self.run_search(init_state, search_method)

    def run_search(self, init_state, search_method):
        print("\nStarting Search")
        self.init_node = SearchNode(init_state, None)
        self.open = [self.init_node]
        self.closed = []
        cnum = 1
        while self.open:
            self.select_node(search_method)
            if self.current_node.goalP(self):
                return self.report_success()
            self.expand(search_method)
            self.closed.append(self.current_node)
            cnum += 1
        return "Search Fails"

    def expand(self, search_method):
        self.successor_nodes = self.current_node.get_successors(self)
        self.vet_successors(search_method)
        if search_method == "depth_first":
            self.open = self.successor_nodes + self.open
        else:
            self.open.extend(self.successor_nodes)

    def vet_successors(self, search_method):
        vslis = []
        if search_method in ["depth_first", "breadth_first", "best_first"]:
            for snode in self.successor_nodes:
                if not self.on_closed(snode) and not self.on_open(snode):
                    vslis.append(snode)
        elif search_method == "branch_and_bound":
            # implementation
            pass
        elif search_method == "A_star":
            # implementation
            pass

        self.successor_nodes = vslis

    def on_closed(self, new_node):
        for closed_node in self.closed:
            if new_node.same_state(closed_node):
                return True
        return False

    def on_open(self, new_node):
        for open_node in self.open:
            if new_node.same_state(open_node):
                return True
        return False

    def select_node(self, search_method):
        if search_method == "depth_first":
            self.current_node = self.open.pop()
        elif search_method == "breadth_first":
            self.current_node = self.open.pop(0)
        else:
            raise ValueError(f"Unsupported search method: {search_method}")

    def report_success(self):
        n = self.current_node
        buf = [str(n)]
        plen = 1
        while n.parent is not None:
            n = n.parent
            buf.insert(0, str(n))
            plen += 1

        print("Search Succeeds")
        print(f"Efficiency {plen / len(self.closed)}")
        print(f"Nodes visited: {len(self.closed) + 1}")
        print("Solution Path:")
        return "\n".join(buf)


# Placeholder for the missing 'search' module
class Search:
    pass


class SearchState:
    pass


class BingoState(SearchState):
    def __init__(self):
        self.grid = [[-1 for _ in range(9)] for _ in range(18)]
        self.unassigned = [(i, j) for i in range(18) for j in range(9)]

    def goalP(self, searcher):
        if self.unassigned:
            return False

        # Check row sums
        for row in self.grid:
            if sum(row) != 5:
                return False

        # Check column sums
        for j in range(9):
            col_sum = sum(row[j] for row in self.grid)
            if j == 0 and col_sum != 9:
                return False
            if j == 8 and col_sum != 11:
                return False
            if 0 < j < 8 and col_sum != 10:
                return False

        # Check column triplets
        return self.check_triplets()

    def check_triplets(self):
        for j in range(9):
            col_sums = [sum(self.grid[i][j] for i in range(0, 18, 3))
                        for i in range(3)]
            if j == 0 and col_sums != [1, 1, 1, 2, 2, 2]:
                return False
            if j == 8 and col_sums != [2, 2, 2, 2, 2, 1]:
                return False
            if 0 < j < 8 and col_sums != [2, 2, 1, 2, 2]:
                return False
        return True

    def get_successors(self, searcher):
        successors = []
        shuffle(self.unassigned)

        for i, j in self.unassigned:
            for val in [0, 1]:
                state = self.copy()
                state.assign(i, j, val)
                if self.is_consistent(state, i, j):
                    successors.append(state)

        return successors

    def is_consistent(self, state, i, j):
        # Check row and column sums
        row_sum = sum(state.grid[i])
        col_sum = sum(row[j] for row in state.grid)
        if row_sum > 5 or col_sum > 11:
            return False

        return True

    def assign(self, i, j, val):
        self.grid[i][j] = val
        self.unassigned.remove((i, j))

    def copy(self):
        state = BingoState()
        state.grid = [row[:] for row in self.grid]
        state.unassigned = self.unassigned[:]
        return state

    def same_state(self, other):
        return self.grid == other.grid

    def cost_from(self, from_state):
        return 1

    def difference(self, goal):
        return sum(1 for i in range(18) for j in range(9) if self.grid[i][j] != goal.grid[i][j])


from random import shuffle


class BingoSearch(Search):

    def __init__(self):
        super().__init__()

    def run_search(self, init_state, search_method):
        print("\nStarting Search")
        self.init_node = SearchNode(init_state, None)
        self.open = [self.init_node]
        self.closed = []
        cnum = 1
        while self.open:
            self.select_node(search_method)
            if self.current_node.goalP(self):
                return self.report_success()
            self.expand(search_method)
            self.closed.append(self.current_node)
            cnum += 1
        return "Search Fails"

    def expand(self, search_method):
        self.successor_nodes = self.current_node.get_successors(self)
        self.vet_successors(search_method)
        if search_method == "depth_first":
            self.open = self.successor_nodes + self.open
        else:
            self.open.extend(self.successor_nodes)

    def vet_successors(self, search_method):
        vslis = []
        if search_method in ["depth_first", "breadth_first", "best_first"]:
            for snode in self.successor_nodes:
                if not self.on_closed(snode) and not self.on_open(snode):
                    vslis.append(snode)
        elif search_method == "branch_and_bound":
            # implementation
            pass
        elif search_method == "A_star":
            # implementation
            pass

        self.successor_nodes = vslis

    def on_closed(self, new_node):
        for closed_node in self.closed:
            if new_node.same_state(closed_node):
                return True
        return False

    def on_open(self, new_node):
        for open_node in self.open:
            if new_node.same_state(open_node):
                return True
        return False

    def select_node(self, search_method):
        if search_method == "depth_first":
            self.current_node = self.open.pop()
        elif search_method == "breadth_first":
            self.current_node = self.open.pop(0)
        else:
            raise ValueError(f"Unsupported search method: {search_method}")

    def report_success(self):
        n = self.current_node
        buf = [str(n)]
        plen = 1
        while n.parent is not None:
            n = n.parent
            buf.insert(0, str(n))
            plen += 1

        print("Search Succeeds")
        print(f"Efficiency {plen / len(self.closed)}")
        print(f"Nodes visited: {len(self.closed) + 1}")
        print("Solution Path:")
        return "\n".join(buf)


if __name__ == "__main__":
    init_state = BingoState()
    searcher = BingoSearch()
    solution = searcher.run_search(init_state, 'depth_first')

    if solution:
        print(solution)
    else:
        print("No solution found.")




Starting Search


KeyboardInterrupt: ignored

In [ ]:
from constraint import Problem, AllDifferentConstraint, ExactSumConstraint
import random

class BingoState:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        self.grid = [[-1] * columns for _ in range(rows)]
        self.unassigned = [(i, j) for i in range(rows) for j in range(columns)]
        self.col_sums = [0] * columns

    def goal_reached(self):
        if self.unassigned:
            return False

        if not self.check_row_sums() or not self.check_column_sums() or not self.check_column_triplets():
            return False

        return True

    def check_row_sums(self):
        return all(sum(row) == 5 for row in self.grid)

    def check_column_sums(self):
        for j in range(self.columns):
            if j == 0 and self.col_sums[j] != 9:
                return False
            if j == self.columns - 1 and self.col_sums[j] != 11:
                return False
            if 0 < j < self.columns - 1 and self.col_sums[j] != 10:
                return False
        return True

    def check_column_triplets(self):
        for j in range(self.columns):
            col_sums = [sum(self.grid[i][j] for i in range(0, self.rows, 3)) for i in range(3)]
            if j == 0 and col_sums != [1, 1, 1, 2, 2, 2]:
                return False
            if j == self.columns - 1 and col_sums != [2, 2, 2, 2, 2, 1]:
                return False
            if 0 < j < self.columns - 1 and col_sums != [2, 2, 1, 2, 2]:
                return False
        return True

    def get_successors(self):
        successors = []
        random.shuffle(self.unassigned)

        for i, j in self.unassigned:
            for val in [0, 1]:
                state = self.copy()
                state.assign(i, j, val)
                if state.is_consistent(i, j):
                    successors.append(state)

        return successors

    def is_consistent(self, i, j):
        row_sum = sum(self.grid[i])
        col_sum = sum(row[j] for row in self.grid)
        return row_sum <= 5 and col_sum <= 11

    def assign(self, i, j, val):
        self.grid[i][j] = val
        self.unassigned.remove((i, j))
        self.col_sums[j] += val

    def copy(self):
        state = BingoState(self.rows, self.columns)
        state.grid = [row[:] for row in self.grid]
        state.unassigned = self.unassigned[:]
        state.col_sums = self.col_sums[:]
        return state

    def same_state(self, other):
        return self.grid == other.grid

    def cost_from(self, from_state):
        return 1

    def difference(self, goal):
        return sum(1 for i in range(self.rows) for j in range(self.columns) if self.grid[i][j] != goal.grid[i][j])

class BingoProblem:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        self.problem = Problem()

    def add_constraints(self):
        for col in range(self.columns):
            for row in range(self.rows):
                self.problem.addVariable((row, col), [0, 1])

        for col in range(self.columns):
            unbound_vars = [(row, col) for row in range(self.rows) if (row, col) in self.problem._variables]
            self.problem.addConstraint(ExactSumConstraint(5), unbound_vars)

        for col in range(self.columns - 1, self.columns - 11, -1):
            unbound_vars = [(row, col) for row in range(self.rows) if (row, col) in self.problem._variables]
            self.problem.addConstraint(ExactSumConstraint(5), unbound_vars)

        for col in range(1, self.columns - 1):
            unbound_vars = [(row, col) for row in range(self.rows) if (row, col) in self.problem._variables]
            self.problem.addConstraint(ExactSumConstraint(5), unbound_vars)

        for row in range(0, self.rows, 3):
            for col in range(self.columns):
                if col % 3 == 0:
                    self.problem.addConstraint(AllDifferentConstraint(), [row, row + 1, row + 2, col])

        for col in range(self.columns):
            if col == 0:
                self.problem.addConstraint(self.triplet_sum_constraint(3, 2), range(0, 9, 3))
                self.problem.addConstraint(self.triplet_sum_constraint(3, 1), range(1, 9, 3))
            elif col == self.columns - 1:
                self.problem.addConstraint(self.triplet_sum_constraint(5, 2), range(0, self.rows, 3))
                self.problem.addConstraint(self.triplet_sum_constraint(1, 1), range(2, self.rows, 3))
            else:
                self.problem.addConstraint(self.triplet_sum_constraint(4, 2), range(0, self.rows, 3))
                self.problem.addConstraint(self.triplet_sum_constraint(2, 1), range(1, self.rows, 3))

    def triplet_sum_constraint(self, total, count):
        def constraint(*args):
            return args.count(1) == total or args.count(1) == count
        return constraint

    def random_order(self, unbound_vars):
        random.shuffle(unbound_vars)
        return unbound_vars

    def generate_successor(self, state):
        unbound_vars = [(i, j) for i in range(self.rows) for j in range(self.columns) if state.grid[i][j] == -1]
        if not unbound_vars:
            return None  # All variables are bound, no successor

        selected_var = random.choice(self.random_order(unbound_vars))
        successor = state.copy()

        # Randomly order the values 0 and 1 for the selected unbound variable
        successor.assign(selected_var[0], selected_var[1], random.choice([0, 1]))

        return successor

    def look_ahead(self, state):
        # Check look-ahead conditions to reduce backtracking
        # You can customize the look-ahead conditions based on your requirements
        return True  # Placeholder, implement your specific conditions

    def depth_first_search(self, state):
        if not state or state.goal_reached():
            return state

        successors = [self.generate_successor(state) for _ in range(len(state.unassigned))]

        for s in successors:
            if s and self.look_ahead(s):
                result = self.depth_first_search(s)
                if result:
                    return result

        return None

    def solve_template(self):
        self.add_constraints()

        # Initialize the template with -1 to represent unbound variables
        initial_state = BingoState(self.rows, self.columns)

        solution = self.depth_first_search(initial_state)
        if solution:
            return solution
        return None

    def display_solution(self, solution):
        for row in solution.grid:
            print(" ".join(map(str, row)))

# Example usage
bingo_problem = BingoProblem(rows=18, columns=9)
solution = bingo_problem.solve_template()

if solution:
    print("Bingo Solution:")
    bingo_problem.display_solution(solution)
else:
    print("No solution found.")


In [ ]:
from random import shuffle

class BingoState:
    def __init__(self):
        # Initialize a grid for the Bingo game
        self.grid = [[-1 for _ in range(9)] for _ in range(18)]
        self.unassigned = [(i, j) for i in range(18) for j in range(9)]
        self.col_sums = [0] * 9

    # Check if the goal state is reached for the Bingo problem
    def goal_reached(self):
        if self.unassigned:
            return False

        if not self.check_row_sums() or not self.check_column_sums() or not self.check_column_triplets():
            return False

        return True

    # Check if the row sums are valid
    def check_row_sums(self):
        return all(sum(row) == 5 for row in self.grid)

    # Check if the column sums are valid
    def check_column_sums(self):
        for j in range(9):
            if j == 0 and self.col_sums[j] != 9:
                return False
            if j == 8 and self.col_sums[j] != 11:
                return False
            if 0 < j < 8 and self.col_sums[j] != 10:
                return False
        return True

    # Check if the column triplets are valid
    def check_column_triplets(self):
        for j in range(9):
            col_sums = [sum(self.grid[i][j] for i in range(0, 18, 3)) for i in range(3)]
            if j == 0 and col_sums != [1, 1, 1, 2, 2, 2]:
                return False
            if j == 8 and col_sums != [2, 2, 2, 2, 2, 1]:
                return False
            if 0 < j < 8 and col_sums != [2, 2, 1, 2, 2]:
                return False
        return True

    # Get successors for the current state
    def get_successors(self):
        successors = []
        shuffle(self.unassigned)

        for i, j in self.unassigned:
            for val in [0, 1]:
                state = self.copy()
                state.assign(i, j, val)
                if state.is_consistent(i, j):
                    successors.append(state)

        return successors

    # Check if the assignment is consistent
    def is_consistent(self, i, j):
        row_sum = sum(self.grid[i])
        col_sum = sum(row[j] for row in self.grid)
        return row_sum <= 5 and col_sum <= 11

    # Assign a value to a cell in the grid
    def assign(self, i, j, val):
        self.grid[i][j] = val
        self.unassigned.remove((i, j))
        self.col_sums[j] += val

    # Create a copy of the current state
    def copy(self):
        state = BingoState()
        state.grid = [row[:] for row in self.grid]
        state.unassigned = self.unassigned[:]
        state.col_sums = self.col_sums[:]
        return state

    # Check if the current state is the same as another state
    def same_state(self, other):
        return self.grid == other.grid

    # Calculate the difference between the current state and a goal state
    def difference(self, goal):
        return sum(1 for i in range(18) for j in range(9) if self.grid[i][j] != goal.grid[i][j])


class BingoSearch:
    def run_search(self, init_state, search_method):
        self.init_node = SearchNode(init_state, None)
        self.open = [self.init_node]
        self.closed = set()

        while self.open:
            self.select_node(search_method)

            if self.current_node.state.goal_reached():
                return self.report_success()

            self.expand()

        return "Search Fails"

    def expand(self):
        self.successor_nodes = self.current_node.state.get_successors()
        self.validate_successors()
        self.open.extend(self.successor_nodes)

    def validate_successors(self):
        valid_successors = [s for s in self.successor_nodes if s not in self.closed and s not in self.open]
        self.successor_nodes = valid_successors

    def select_node(self, search_method):
        if search_method == "depth_first":
            self.current_node = self.open.pop()
        elif search_method == "breadth_first":
            self.current_node = self.open.pop(0)
        else:
            raise ValueError(f"Unsupported search method: {search_method}")

    def report_success(self):
        n = self.current_node
        path = [str(n)]
        plen = 1
        while n.parent is not None:
            n = n.parent
            path.insert(0, str(n))
            plen += 1

        print("Search Succeeds")
        print(f"Efficiency {plen / len(self.closed)}")
        print(f"Nodes visited: {len(self.closed) + 1}")
        print("Solution Path:")
        return "\n".join(path)


class SearchNode:
    def __init__(self, state, parent, cost=1):
        self.state = state
        self.parent = parent
        self.cost = cost

    def goal_reached(self):
        return
